In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Matplotlib inline magic command
%matplotlib inline

In [1]:
# Initial imports
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

import numpy as np
import seaborn as sns
from collections import Counter
import psycopg2
from config import db_password
import json
from pandas.io.json import json_normalize
import mysql.connector
from sqlalchemy import create_engine
from os import walk
import time
import psycopg2
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
# Scikit-Learn Model Selection and Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Scikit-Learn Model imports
from sklearn.linear_model import LinearRegression

In [ ]:
# Sci-kit Learn Metrics imports
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [ ]:
# Imbalanced Learn imports
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.over_sampling import RandomOverSampler

In [2]:
# Connect to AWS
def getDBengine1():
    user = 'postgres'
    passw = db_password
    host = 'policekillings.cv95lgysyvwq.us-east-1.rds.amazonaws.com'
    port = 5432
    database = 'PoliceKillings'
    engine_1 = create_engine(f'postgres+psycopg2://' + user + ':' + passw + '@' + host + ':' + str(port) + '/' + database ,  echo=False)
    return engine_1

In [6]:
# Load database into DataFrame
def getData1():
    s_statement = "select * from public.police_killings"
    # df = pd.read_sql(s_statement, con=getDBengine())
    try:
        df = pd.read_sql(s_statement, con=getDBengine1())
    except:
        print('failed')
    return df

In [7]:
df_data = getData1()
df_data.shape
print(df_data.columns)

Index(['Victim_Age', 'Victim_Gender', 'Victim_Race', 'Date', 'City', 'State',
       'County', 'Responsible_Agency', 'Cause_of_death', 'Brief_Description ',
       'Criminal_Charges', 'Mental_Illness', 'Armed_Status', 'Threat_Level ',
       'Fleeing ', 'Body_Camera', 'Geography', 'Encounter_Type ',
       'Initial_Reason_for_Encounter ', 'Call_for_Service'],
      dtype='object')


In [8]:
df_data

,Victim_Age,Victim_Gender,Victim_Race,Date,City,State,County,Responsible_Agency,Cause_of_death,Brief_Description,Criminal_Charges,Mental_Illness,Armed_Status,Threat_Level,Fleeing,Body_Camera,Geography,Encounter_Type,Initial_Reason_for_Encounter,Call_for_Service
0,Unknown,Male,Unknown Race,2021-04-18,Detroit,MI,None,Detroit Police Department,Gunshot,None,No known charges,None,Allegedly Armed,attack,None,None,None,Part 1 Violent Crime,shooting,None
1,Unknown,Male,Unknown Race,2021-04-18,Douglasville,GA,None,Douglasville Police Department,Gunshot,None,No known charges,None,None,None,None,None,None,None,None,None
2,Unknown,Male,Unknown Race,2021-04-18,Fort Worth,TX,None,Fort Worth Police Department,Gunshot,None,No known charges,None,Allegedly Armed,None,None,None,None,Other Non-Violent Offense,carjacking,None
3,Unknown,Male,White,2021-04-18,Burnsville,MN,None,Burnsville Police Department,Gunshot,None,No known charges,None,Allegedly Armed,attack,car,None,None,Other Non-Violent Offense,carjacking,None
4,Unknown,Male,Black,2021-04-17,Winter Haven,FL,None,Winter Haven Police Department,Gunshot,None,No known charges,None,None,None,None,None,None,Domestic disturbance,Domestic disturbance (armed),None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9077,21,Male,White,2013-01-01,Gordon,WI,Douglas,Douglas County Sheriff's Office,Gunshot,Deputies responded to a 911 call of shots fire...,No known charges,Drug or alcohol use,Allegedly Armed,None,None,None,Rural,None,None,None
9078,26,Male,White,2013-01-01,Mankato,MN,Blue Earth,Mankato Department of Public Safety,Taser,Former combat medic Layton was found unconscio...,No known charges,No,Unarmed/Did Not Have Actual Weapon,None,None,None,Suburban,None,None,None
9079,21,Male,Hispanic,2013-01-01,Pueblo,CO,Pueblo,Pueblo Police Department,Gunshot,Christopher was with two others in a car. Pueb...,No known charges,No,Allegedly Armed,None,None,None,Suburban,None,None,None
9080,49,Male,Hispanic,2013-01-01,Farmington,NM,San Juan,Farmington Police Department,Gunshot,An officer responded to a report by Chavez tha...,No known charges,No,Allegedly Armed,None,None,None,Urban,None,None,None


In [ ]:
# Load the data
#file_path = 'Resources/2013-2020_Police_Killings_Revised.xlsx' 
#killings_df = pd.read_excel(file_path)                         
#killings_df

In [ ]:
# Rename columns
killings_df.rename(columns = {"Victim's age" : 'Victim_Age', "Victim's gender" : 'Victim_Gender', 
                              "Victim's race" : 'Victim_Race', 'Date of Incident (month/day/year)' : 'Date',
                              'Agency responsible for death' : 'Responsible_Agency', 'Cause of death' : 'Cause_of_Death',
                              'A brief description of the circumstances surrounding the death' : 'Brief_Description',
                              'Criminal Charges?' : 'Criminal_Charges', 'Symptoms of mental illness?' : 'Mental_Illness', 
                              'Armed/Unarmed Status' : 'Armed_Status', 'Alleged Threat Level (Source: WaPo)' : 'Threat_Level', 
                              'Fleeing (Source: WaPo)' : 'Fleeing', 'Body Camera (Source: WaPo)' : 'Body_Camera', 
                              'Encounter Type (DRAFT)' : 'Encounter_Type', 
                              'Initial Reported Reason for Encounter (DRAFT)' : 'Initial_Reason_for_Encounter',
                              'Call for Service? (DRAFT)' : 'Call_for_Service'}, inplace = True)

killings_df.head()

In [ ]:
# Delete Gender column
del killings_df['Victim_Gender']

In [ ]:
# Find null values
for column in killings_df.columns:
    print(f'Column {column} has {killings_df[column].isnull().sum()} null values')

In [ ]:
# Replace empty/blank values for Age, Threat_Level, Fleeing, Body_Camera, Encounter_Type, Initial_Reason_for_Encounter, 
# Call_for_Service and re-verify null values

# Column: Age
killings_df['Victim_Age'].replace({'Unknown': '0'}, inplace = True) # Possible to fill in 'Unknown' with mean
                                                                    # of column?
# Column: Threat_Level
killings_df['Threat_Level'].replace({None: 'undetermined'}, inplace = True)

# Column: Fleeing
killings_df['Fleeing'].replace({None: 'unknown'}, inplace = True)

# Column: Body_Camera
killings_df['Body_Camera'].replace({None: 'unknown'}, inplace = True)

# Column: Encounter_Type
killings_df['Encounter_Type'].replace({None: 'Unknown'}, inplace = True)

# Column: Initial_Reason_for_Encounter
killings_df['Initial_Reason_for_Encounter'].replace({None: 'unknown'}, inplace = True)

# Column: Call_for_Service
killings_df['Call_for_Service'].replace({None: 'Unavailable'}, inplace = True)

# Verify null values
for column in killings_df.columns:
    print(f'Column {column} has {killings_df[column].isnull().sum()} null values')

In [ ]:
# Drop null values
killings_df = killings_df.dropna(how = 'any')
killings_df = killings_df.reset_index().drop(['index'], axis = 1)
killings_df

In [ ]:
killings_df['Brief_Description'][4532]

In [ ]:
# Inspect column data types
killings_df.dtypes

In [ ]:
# Change Victim_Age data type from object to int
killings_df['Victim_Age'] = killings_df['Victim_Age'].astype(str).astype(int)
killings_df.dtypes

In [ ]:
# Extract year from Date and create new column Year
killings_df['Year'] = pd.DatetimeIndex(killings_df['Date']).year

# Extract month from Date and create new column Month
killings_df['Month'] = pd.DatetimeIndex(killings_df['Date']).month

# Extract day from Date and create new column Day
killings_df['Day'] = pd.DatetimeIndex(killings_df['Date']).day

# Get day of week from Date and create new column Day_of_Week
killings_df['Day_of_Week'] = killings_df['Date'].dt.day_name()

In [ ]:
# Get holiday from Date and create new column Holiday
dr = pd.date_range(start='2013-01-01', end='2021-04-18')
df = pd.DataFrame()
df['Date'] = dr

cal = calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())

killings_df['Holiday'] = killings_df['Date'].dt.date.astype('datetime64').isin(holidays)

In [ ]:
#killings_df

In [ ]:
# Use get_dummies() to transform text values into numerical values
killings_encoded = pd.get_dummies(killings_df, columns = 
                                  ['Cause_of_Death', 'Mental_Illness', 'Armed_Status', 'Threat_Level', 'Fleeing', 'Body_Camera', 
                                   'Geography'])

killings_encoded.head()

# We are keeping race because that is what our model will be running against, right?  Age was converted to an 
# int and Date is now datetime
# Are we deleting the columns that are not in killings_encoded then?  I took notes on this during the tutor
# session, but they do not make sense to me

## Split Data into Training and Testing Sets

In [ ]:
# Create features
X = pd.get_dummies(killings_df.drop(columns = ['race']))

# Create target
y = pd.DataFrame(killings_df['race'])

In [ ]:
X.describe()

In [ ]:
# Check balance of target values
y['race'].value_counts()

In [ ]:
# Train, test, and split model
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state = 78,
                                                    stratify = y)

y_train.shape

In [ ]:
# Creating a StandardScaler instance
scaler = StandardScaler()

# Fitting the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

# Scaling the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# ML model code
# Still need to pick a model

In [ ]:
# Calculate balanced accuracy score
rf_y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test, rf_y_pred)

In [ ]:
# Display confusion matrix
rf_cm = confusion_matrix(y_test, rf_y_pred)
rf_cm

In [ ]:
# Print imbalanced classification report
print(classification_report_imbalanced(y_test, rf_y_pred))

In [ ]:
# List feature sorted in descending order by feature importance - is this needed?
sorted(zip(rf_model.feature_importances_, X.columns), reverse = True)

In [ ]:
# Model prediction
print(rf_y_pred)